In [79]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_column',500)
df=pd.read_csv('data/player_season_stats.csv').drop('Unnamed: 0',axis=1)
# df=df.set_index(['player', 'year'])

def extract_rankings_from_text(text):
    try:
        text_string=re.findall("[0-9]+",text)
        rankings=list(map(int,text_string))
        avg_rankings=sum(rankings)/len(rankings)
        return int(avg_rankings)
    except TypeError:
        return np.NaN

In [39]:
df['assigned_draft_rank']=df.apply(lambda x: extract_rankings_from_text(x.rankings),axis=1)
df[~df['rankings'].isnull()][['rankings','assigned_draft_rank']].sort_values('assigned_draft_rank')


rankings  \
player               year                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
Alexis Lafrenière    2019-2020  Ranked #1 by HOCKEYPROSPECT.COM                                                                                                                                                                                     Ranked #1 by FUTURE CONSIDERATIONS                                                                                                                                                                                     Ranked #1 by ISS HOCKEY                                              ...   
                     2017-2018  Ranked #1 by HOCKEYPROSPECT.COM                                                                                                                                                                                     Ranked #1 by FUTURE CONSIDERATIONS                                                                                                                                                                                     Ranked #1 by ISS HOCKEY                                              ...   
                     2018-2019  Ranked #1 by HOCKEYPROSPECT.COM                                                                                                                                                                                     Ranked #1 by FUTURE CONSIDERATIONS                                                                                                                                                                                     Ranked #1 by ISS HOCKEY                                              ...   
Quinton Byfield      2019-2020  Ranked #2 by HOCKEYPROSPECT.COM                                                                                                                                                                                     Ranked #2 by FUTURE CONSIDERATIONS                                                                                                                                                                                     Ranked #4 by ISS HOCKEY                                              ...   
                     2018-2019  Ranked #2 by HOCKEYPROSPECT.COM                                                                                                                                                                                     Ranked #2 by FUTURE CONSIDERATIONS                                                                                                                                                                                     Ranked #4 by ISS HOCKEY                                              ...   
...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ...   
Frankie Carogioiello 2019-2020                                                                                                                                                                                                                     

In [75]:
df['assigned_draft_round'] = pd.cut(df.assigned_draft_rank, 
                                    bins=[p for p in range(1, 219, 31)],
                                    labels=[r for r in range(1, 8, 1)],
                                    right=False)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [69]:
df[df.player == 'Alexis Lafrenière']

,playerid,player,position,league,year,team,teamids,gp,g,a,tp,gpg,apg,ppg,perc_team_g,perc_team_a,perc_team_tp,date_of_birth,place_of_birth,nation,height,weight,shoots,rights,under_contract,draft_year,draft_round,draft_pick,draft_team,shortname,status,nhl_draft,rankings,catches,draft_year_eligible,load_date,start_year,end_year,season_age,real_season_age
2282,296697,Alexis Lafrenière,{LW},QMJHL,2017-2018,['RIM'],['889'],60.0,42.0,38.0,80.0,0.700000,0.633333,1.333333,0.205,0.108,0.143,2001-10-11,"Saint-Eustache, QC, CAN",Canada,186.0,87.0,L,NaN,NaN,NaN,NaN,NaN,NaN,alexis-lafreniere,NaN,Top Prospect eligible for the 2020 NHL Entry Draft,Ranked #1 by HOCKEYPROSPECT.COM Ranked #1 by FUTURE CONSIDERATIONS Ranked #1 by ISS HOCKEY ...,NaN,2020,2020-06-03,2017,2018,17,17.22
10182,296697,Alexis Lafrenière,{LW},QMJHL,2018-2019,['RIM'],['889'],61.0,37.0,68.0,105.0,0.606557,1.114754,1.721311,0.153,0.162,0.158,2001-10-11,"Saint-Eustache, QC, CAN",Canada,186.0,87.0,L,NaN,NaN,NaN,NaN,NaN,NaN,alexis-lafreniere,NaN,Top Prospect eligible for the 2020 NHL Entry Draft,Ranked #1 by HOCKEYPROSPECT.COM Ranked #1 by FUTURE CONSIDERATIONS Ranked #1 by ISS HOCKEY ...,NaN,2020,2020-06-03,2018,2019,18,18.22
26489,296697,Alexis Lafrenière,{LW},QMJHL,2019-2020,['RIM'],['889'],52.0,35.0,77.0,112.0,0.673077,1.480769,2.153846,0.171,0.222,0.203,2001-10-11,"Saint-Eustache, QC, CAN",Canada,186.0,87.0,L,NaN,NaN,NaN,NaN,NaN,NaN,alexis-lafreniere,NaN,Top Prospect eligible for the 2020 NHL Entry Draft,Ranked #1 by HOCKEYPROSPECT.COM Ranked #1 by FUTURE CONSIDERATIONS Ranked #1 by ISS HOCKEY ...,NaN,2020,2020-06-03,2019,2020,19,19.22


In [80]:
def draft_current_year(df):
    
    df = df.set_index(['playerid', 'player', 'year'])
    
    df['assigned_draft_rank'] = df.apply(lambda x: extract_rankings_from_text(x.rankings),axis=1)
    
    df['assigned_draft_round'] = pd.cut(df.assigned_draft_rank, 
                                    bins=[p for p in range(1, 219, 31)],
                                    labels=[r for r in range(1, 8, 1)],
                                    right=False)
    
    df['draft_round'] = np.where(df.draft_round.isnull(),
                                df.assigned_draft_rank,
                                df.draft_round)  
    
    df['draft_pick'] = np.where(df.draft_pick.isnull(),
                               df.assigned_draft_round,
                               df.draft_pick)  
    
    return df.reset_index()
    

In [81]:
df = draft_current_year(df)
df[df.player == 'Alexis Lafrenière']

,playerid,player,year,position,league,team,teamids,gp,g,a,tp,gpg,apg,ppg,perc_team_g,perc_team_a,perc_team_tp,date_of_birth,place_of_birth,nation,height,weight,shoots,rights,under_contract,draft_year,draft_round,draft_pick,draft_team,shortname,status,nhl_draft,rankings,catches,draft_year_eligible,load_date,start_year,end_year,season_age,real_season_age,assigned_draft_rank,assigned_draft_round
2282,296697,Alexis Lafrenière,2017-2018,{LW},QMJHL,['RIM'],['889'],60.0,42.0,38.0,80.0,0.700000,0.633333,1.333333,0.205,0.108,0.143,2001-10-11,"Saint-Eustache, QC, CAN",Canada,186.0,87.0,L,NaN,NaN,NaN,1.0,1.0,NaN,alexis-lafreniere,NaN,Top Prospect eligible for the 2020 NHL Entry Draft,Ranked #1 by HOCKEYPROSPECT.COM Ranked #1 by FUTURE CONSIDERATIONS Ranked #1 by ISS HOCKEY ...,NaN,2020,2020-06-03,2017,2018,17,17.22,1.0,1
10182,296697,Alexis Lafrenière,2018-2019,{LW},QMJHL,['RIM'],['889'],61.0,37.0,68.0,105.0,0.606557,1.114754,1.721311,0.153,0.162,0.158,2001-10-11,"Saint-Eustache, QC, CAN",Canada,186.0,87.0,L,NaN,NaN,NaN,1.0,1.0,NaN,alexis-lafreniere,NaN,Top Prospect eligible for the 2020 NHL Entry Draft,Ranked #1 by HOCKEYPROSPECT.COM Ranked #1 by FUTURE CONSIDERATIONS Ranked #1 by ISS HOCKEY ...,NaN,2020,2020-06-03,2018,2019,18,18.22,1.0,1
26489,296697,Alexis Lafrenière,2019-2020,{LW},QMJHL,['RIM'],['889'],52.0,35.0,77.0,112.0,0.673077,1.480769,2.153846,0.171,0.222,0.203,2001-10-11,"Saint-Eustache, QC, CAN",Canada,186.0,87.0,L,NaN,NaN,NaN,1.0,1.0,NaN,alexis-lafreniere,NaN,Top Prospect eligible for the 2020 NHL Entry Draft,Ranked #1 by HOCKEYPROSPECT.COM Ranked #1 by FUTURE CONSIDERATIONS Ranked #1 by ISS HOCKEY ...,NaN,2020,2020-06-03,2019,2020,19,19.22,1.0,1
